In [6]:
import ffmpeg

probe = ffmpeg.probe('input1.mp4')
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')

width = int(video_info['width'])
height = int(video_info['height'])
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
duration = float(video_info['duration'])
fps = 30

print(f"Width: {width}, Height: {height}, FPS: {fps}, duration: {duration} seconds")

Width: 1920, Height: 1080, FPS: 30, duration: 1800.04825 seconds


In [ ]:
import ffmpeg

probe = ffmpeg.probe('input1.mp4')


print(f"Duration: {duration} seconds")

Duration: 1800.04825 seconds


In [ ]:
import cv2

video = cv2.VideoCapture('video.mp4')

fps = video.get(cv2.CAP_PROP_FPS)
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps

video.release()

print(f"Duration: {duration} seconds")

In [7]:
from convert_to_audio import convert_video

convert_video("input1.mp4", target_height=360)  # 360p

ffmpeg version 8.0 Copyright (c) 2000-2025 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.6)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/8.0_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --e

Video successfully converted to 360p: input1_360p.mp4


[out#0/mp4 @ 0x12be28b80] video:49072KiB audio:28001KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 1.738721%
frame=43158 fps=503 q=-1.0 Lsize=   78414KiB time=00:29:59.96 bitrate= 356.9kbits/s speed=  21x elapsed=0:01:25.82    
[libx264 @ 0x12d0106a0] frame I:288   Avg QP:17.78  size: 41496
[libx264 @ 0x12d0106a0] frame P:13305 Avg QP:21.58  size:  2078
[libx264 @ 0x12d0106a0] frame B:29565 Avg QP:29.01  size:   360
[libx264 @ 0x12d0106a0] consecutive B-frames:  5.0%  9.0%  6.3% 79.7%
[libx264 @ 0x12d0106a0] mb I  I16..4:  3.3% 53.9% 42.8%
[libx264 @ 0x12d0106a0] mb P  I16..4:  0.1%  0.8%  0.4%  P16..4: 13.6%  7.3%  5.5%  0.0%  0.0%    skip:72.3%
[libx264 @ 0x12d0106a0] mb B  I16..4:  0.0%  0.1%  0.0%  B16..8: 12.5%  1.6%  0.4%  direct: 0.3%  skip:85.1%  L0:39.9% L1:52.2% BI: 7.9%
[libx264 @ 0x12d0106a0] 8x8 transform intra:57.6% inter:62.1%
[libx264 @ 0x12d0106a0] coded y,uvDC,uvAC intra: 86.3% 87.3% 66.7% inter: 3.9% 2.8% 0.2%
[libx264 @ 0x12d0106a0] i16 v,

The quick brown fox leaps over the sleepy dog while bright birds chatter in tall green trees under a warm golden sunset, as a gentle breeze carries the scent of fresh grass across the quiet countryside.

In [2]:
pwd

'/Users/cohlem/Projects/Experimentation/remotion_backend/subtitle_generator'

In [8]:
# Update subtitle_generator/utils/hybrid_line_divider.py
import re
import logging
import random
from typing import List, Dict, Optional, Tuple, Any, Literal
from models import (
    GroupWithHighlight, 
    SubtitleGroup, 
    SubtitleLine,
    FontConfig, GroupDivisionWithHighlights, SubtitleTimeline
)

logger = logging.getLogger(__name__)





class HybridLineDivider:
    """
    Rule-based line divider for hybrid subtitle processing.
    Divides groups into lines using highlight word as anchor.
    Supports configurable fonts per style.
    """
    
    def __init__(self, max_words_per_line: int = 3, font_config: Optional[FontConfig] = None):
        self.max_words = max_words_per_line
        self.font_config = font_config or FontConfig()  # Default to normal only
        random.seed()  # Initialize random for font selection
    
    def divide_group(self, group: GroupWithHighlight) -> SubtitleGroup:
        """
        Divide a single group into lines based on highlight word and font config.
        """
        text = group.group_text
        highlight = group.highlight_word
        
        # If no highlight or highlight not in text, treat as normal group
        if not highlight or highlight not in text:
            return self._divide_without_highlight(text)
        
        # Split text into words preserving order
        words = text.split()
        
        # Find highlight position
        try:
            clean_words = [self._clean_word(w) for w in words]
            clean_highlight = self._clean_word(highlight)
            highlight_idx = clean_words.index(clean_highlight)
        except ValueError:
            logger.warning(f"Highlight word '{highlight}' not found in '{text}'")
            return self._divide_without_highlight(text)
        
        # Split into before and after highlight
        before_words = words[:highlight_idx]
        after_words = words[highlight_idx + 1:]
        
        # Build lines
        lines = []
        
        # Get fonts from config
        highlight_font = self.font_config.get_highlight_font()
        supporting_fonts = self.font_config.get_supporting_fonts()  # Get list of available supporting fonts
        

        print('supporting fonts', supporting_fonts)
        # Handle before highlight - split into chunks of max_words
        # Alternate between available supporting fonts
        if before_words:
            before_chunks = self._chunk_words(before_words, self.max_words)
            for i, chunk in enumerate(before_chunks):
                # Cycle through supporting fonts: 0, 1, 0, 1, etc.
                font_idx = i % len(supporting_fonts)
                print('before words chunk', chunk, supporting_fonts[font_idx])
                lines.append(SubtitleLine(
                    text=" ".join(chunk),
                    font_type=supporting_fonts[font_idx]
                ))
        
        # Add highlight line with configured highlight font (bold or italic)
        lines.append(SubtitleLine(
            text=highlight,
            font_type=highlight_font
        ))
        
        # Handle after highlight - split into chunks of max_words
        # Continue alternating pattern from where we left off
        if after_words:
            after_chunks = self._chunk_words(after_words, self.max_words)
            start_idx = len(before_chunks) % len(supporting_fonts) if before_words else 0
            for i, chunk in enumerate(after_chunks):
                font_idx = (start_idx + i) % len(supporting_fonts)

                print('after words chunk', chunk, supporting_fonts[font_idx])
                lines.append(SubtitleLine(
                    text=" ".join(chunk),
                    font_type=supporting_fonts[font_idx]
                ))
        
        # Ensure we don't exceed 3 lines total
        lines = self._optimize_lines(lines, supporting_fonts)
        
        return SubtitleGroup(
            group_text=text,
            lines=lines
        )
    
    def _clean_word(self, word: str) -> str:
        """Remove punctuation for matching."""
        return re.sub(r'[^\w\s]', '', word.lower()).strip()
    
    def _chunk_words(self, words: List[str], max_size: int) -> List[List[str]]:
        """Split word list into chunks of max_size."""
        if not words:
            return []
        
        chunks = []
        current_chunk = []
        
        for word in words:
            if len(current_chunk) >= max_size:
                chunks.append(current_chunk)
                current_chunk = []
            current_chunk.append(word)
        
        if current_chunk:
            chunks.append(current_chunk)
        
        return chunks
    
    def _divide_without_highlight(self, text: str) -> SubtitleGroup:
        """Divide group without highlight word (all supporting fonts)."""
        words = text.split()
        
        # Simple chunking
        chunks = self._chunk_words(words, self.max_words)
        
        supporting_fonts = self.font_config.get_supporting_fonts()
        
        # Alternate between supporting fonts
        lines = []
        for i, chunk in enumerate(chunks):
            font_idx = i % len(supporting_fonts)
            lines.append(SubtitleLine(
                text=" ".join(chunk), 
                font_type=supporting_fonts[font_idx]
            ))
        
        return SubtitleGroup(group_text=text, lines=lines)
    
    def _optimize_lines(self, lines: List[SubtitleLine], supporting_fonts: List[str]) -> List[SubtitleLine]:
        """
        Ensure we have at most 3 lines.
        If more than 3, merge supporting lines intelligently while alternating fonts.
        """
        if len(lines) <= 3:
            return lines
        
        # Find highlight line index
        highlight_font = self.font_config.get_highlight_font()
        highlight_idx = None
        
        for i, line in enumerate(lines):
            if line.font_type == highlight_font:
                highlight_idx = i
                break
        
        if highlight_idx is None:
            # No highlight found, just take first 3 chunks with alternating fonts
            result = []
            for i in range(min(3, len(lines))):
                font_idx = i % len(supporting_fonts)
                result.append(SubtitleLine(text=lines[i].text, font_type=supporting_fonts[font_idx]))
            return result
        
        # Strategy: Keep highlight line, merge others
        before = lines[:highlight_idx]
        highlight = lines[highlight_idx]
        after = lines[highlight_idx + 1:]
        
        # Merge strategy: reduce to max 1 line before and 1 line after
        while len(before) > 1 and len(after) > 1:
            if len(before) >= len(after):
                # Merge last two before lines - use the font of the first one to maintain alternation pattern
                merged_text = before[-2].text + " " + before[-1].text
                before[-2] = SubtitleLine(text=merged_text, font_type=before[-2].font_type)
                before.pop(-1)
            else:
                # Merge first two after lines
                merged_text = after[0].text + " " + after[1].text
                after[0] = SubtitleLine(text=merged_text, font_type=after[0].font_type)
                after.pop(1)
        
        # If still over 3 lines, merge all extras into adjacent lines
        while len(before) + 1 + len(after) > 3:
            if len(before) > 1:
                # Merge last two before
                merged_text = before[-2].text + " " + before[-1].text
                before[-2] = SubtitleLine(text=merged_text, font_type=before[-2].font_type)
                before.pop(-1)
            elif len(after) > 1:
                # Merge first two after
                merged_text = after[0].text + " " + after[1].text
                after[0] = SubtitleLine(text=merged_text, font_type=after[0].font_type)
                after.pop(1)
            else:
                break
        
        # Ensure alternating fonts for final result if we have 2 supporting lines
        result = before + [highlight] + after
        if len(result) == 3 and len(supporting_fonts) >= 2:
            # Check if both supporting lines have same font
            supporting_indices = [i for i, line in enumerate(result) if line.font_type != highlight_font]
            if len(supporting_indices) == 2:
                idx1, idx2 = supporting_indices
                if result[idx1].font_type == result[idx2].font_type:
                    # They have same font, change second one to alternate
                    current_font = result[idx1].font_type
                    alternate_font = supporting_fonts[1] if current_font == supporting_fonts[0] else supporting_fonts[0]
                    result[idx2] = SubtitleLine(text=result[idx2].text, font_type=alternate_font)
        
        return result
    
    def divide_groups(self, groups: List[GroupWithHighlight]) -> List[SubtitleGroup]:
        """Process multiple groups."""
        return [self.divide_group(g) for g in groups]


class HybridPostProcessor:
    """
    Post-process groups with highlights, splitting oversized groups
    and correctly assigning highlight words to split parts.
    """
    
    def __init__(self, max_words_per_group: int = 8):
        self.max_words = max_words_per_group
    
    def process(
        self, 
        division: 'GroupDivisionWithHighlights'
    ) -> List[GroupWithHighlight]:
        """
        Split oversized groups and assign highlight words correctly.
        """
        # from subtitle_generator.models import GroupDivisionWithHighlights
        
        result = []
        
        for group in division.groups:
            word_count = len(group.group_text.split())
            
            if word_count <= self.max_words:
                result.append(group)
            else:
                # Split and distribute highlight
                split_groups = self._split_group_with_highlight(group)
                result.extend(split_groups)
        
        return result
    
    def _split_group_with_highlight(
        self, 
        group: GroupWithHighlight
    ) -> List[GroupWithHighlight]:
        """
        Split oversized group and determine which part gets the highlight.
        """
        words = group.group_text.split()
        highlight = group.highlight_word
        
        # Find highlight position
        try:
            clean_words = [re.sub(r'[^\w]', '', w.lower()) for w in words]
            clean_highlight = re.sub(r'[^\w]', '', highlight.lower()) if highlight else ""
            highlight_idx = clean_words.index(clean_highlight) if clean_highlight else -1
        except ValueError:
            highlight_idx = -1
        
        # Calculate split point
        total_words = len(words)
        mid_point = total_words // 2
        
        # Adjust split to not break in middle of a potential highlight
        split_point = mid_point
        
        # Split words
        first_half = words[:split_point]
        second_half = words[split_point:]
        
        first_text = " ".join(first_half)
        second_text = " ".join(second_half)
        
        # Determine which half gets the highlight
        first_highlight = None
        second_highlight = None
        
        if highlight_idx >= 0 and highlight_idx < split_point:
            # Highlight is in first half
            first_highlight = highlight
            # Try to find a secondary highlight in second half (longest word)
            if len(second_half) > 1:
                second_highlight = max(second_half, key=len)
        elif highlight_idx >= split_point:
            # Highlight is in second half
            second_highlight = highlight
            # Try to find secondary in first half
            if len(first_half) > 1:
                first_highlight = max(first_half, key=len)
        else:
            # No highlight found, assign longest words
            if len(first_half) > 1:
                first_highlight = max(first_half, key=len)
            if len(second_half) > 1:
                second_highlight = max(second_half, key=len)
        
        result = []
        
        if first_half:
            result.append(GroupWithHighlight(
                group_text=first_text,
                highlight_word=first_highlight
            ))
        
        if second_half:
            result.append(GroupWithHighlight(
                group_text=second_text,
                highlight_word=second_highlight
            ))
        
        return result
    
    def process_divisions(
        self, 
        divisions: List['GroupDivisionWithHighlights']
    ) -> List[List[GroupWithHighlight]]:
        """Process multiple divisions (one per chunk)."""
        return [self.process(d) for d in divisions]

In [9]:
divisions_with_highlights = [GroupDivisionWithHighlights(groups=[GroupWithHighlight(group_text="one I've met", highlight_word='met'), 
GroupWithHighlight(group_text="So in this video I am going to show you yes i am hehe something you have and im testing it a lot more never's seen", highlight_word=None), 
GroupWithHighlight(group_text='I want to share', highlight_word='share'), 
GroupWithHighlight(group_text='10 of my top tips', highlight_word='tips'), 
GroupWithHighlight(group_text='for reading more consistently', highlight_word='reading'), 
GroupWithHighlight(group_text='And if you follow', highlight_word='follow'), 
GroupWithHighlight(group_text='the tips in this video', highlight_word='tips'), 
GroupWithHighlight(group_text="then there's a high chance", highlight_word='chance'), 
GroupWithHighlight(group_text='you might even be able', highlight_word='able'), 
GroupWithHighlight(group_text='to read a hundred books a year', highlight_word='read'), 
GroupWithHighlight(group_text='but', highlight_word=None)])]



post_processor = HybridPostProcessor(max_words_per_group=8)
line_divider = HybridLineDivider(max_words_per_line=3, font_config=FontConfig(normal=True, bold=True, italic=True))

processed_groups_per_chunk = post_processor.process_divisions(divisions_with_highlights)

processed_groups_per_chunk

[[GroupWithHighlight(group_text="one I've met", highlight_word='met'),
  GroupWithHighlight(group_text='So in this video I am going to show you yes i am', highlight_word='video'),
  GroupWithHighlight(group_text="hehe something you have and im testing it a lot more never's seen", highlight_word='something'),
  GroupWithHighlight(group_text='I want to share', highlight_word='share'),
  GroupWithHighlight(group_text='10 of my top tips', highlight_word='tips'),
  GroupWithHighlight(group_text='for reading more consistently', highlight_word='reading'),
  GroupWithHighlight(group_text='And if you follow', highlight_word='follow'),
  GroupWithHighlight(group_text='the tips in this video', highlight_word='tips'),
  GroupWithHighlight(group_text="then there's a high chance", highlight_word='chance'),
  GroupWithHighlight(group_text='you might even be able', highlight_word='able'),
  GroupWithHighlight(group_text='to read a hundred books a year', highlight_word='read'),
  GroupWithHighlight(gro

In [10]:
timelines = []
for chunk_idx, chunk_groups in enumerate(processed_groups_per_chunk):
    subtitle_groups = line_divider.divide_groups(chunk_groups)
    timeline = SubtitleTimeline(timeline=subtitle_groups)
    timelines.append(timeline)
    logger.info(f"[Chunk {chunk_idx}] Divided into {len(subtitle_groups)} subtitle groups")

supporting fonts ['normal', 'italic']
before words chunk ['one', "I've"] normal
supporting fonts ['normal', 'italic']
before words chunk ['So', 'in', 'this'] normal
after words chunk ['I', 'am', 'going'] italic
after words chunk ['to', 'show', 'you'] normal
after words chunk ['yes', 'i', 'am'] italic
supporting fonts ['normal', 'italic']
before words chunk ['hehe'] normal
after words chunk ['you', 'have', 'and'] italic
after words chunk ['im', 'testing', 'it'] normal
after words chunk ['a', 'lot', 'more'] italic
after words chunk ["never's", 'seen'] normal
supporting fonts ['normal', 'italic']
before words chunk ['I', 'want', 'to'] normal
supporting fonts ['normal', 'italic']
before words chunk ['10', 'of', 'my'] normal
before words chunk ['top'] italic
supporting fonts ['normal', 'italic']
before words chunk ['for'] normal
after words chunk ['more', 'consistently'] italic
supporting fonts ['normal', 'italic']
before words chunk ['And', 'if', 'you'] normal
supporting fonts ['normal', '

In [7]:
timelines[0].timeline

[SubtitleGroup(group_text="one I've met", lines=[SubtitleLine(text="one I've", font_type='normal'), SubtitleLine(text='met', font_type='bold')], id=None, start=None, end=None),
 SubtitleGroup(group_text='So in this video I am going to show you yes i am', lines=[SubtitleLine(text='So in this', font_type='normal'), SubtitleLine(text='video', font_type='bold'), SubtitleLine(text='I am going to show you yes i am', font_type='italic')], id=None, start=None, end=None),
 SubtitleGroup(group_text="hehe something you have and im testing it a lot more never's seen", lines=[SubtitleLine(text='hehe', font_type='normal'), SubtitleLine(text='something', font_type='bold'), SubtitleLine(text="you have and im testing it a lot more never's seen", font_type='italic')], id=None, start=None, end=None),
 SubtitleGroup(group_text='I want to share', lines=[SubtitleLine(text='I want to', font_type='normal'), SubtitleLine(text='share', font_type='bold')], id=None, start=None, end=None),
 SubtitleGroup(group_tex